# kaggle - Learn: Intermediate Machine Learning
- https://www.kaggle.com/learn/intermediate-machine-learning
> JM-Future: mk a program to obtain a mini dataset from a big dataset
mantaining cols names and a prportional number of NaN in every col?, ex. to conver melb_data to min_melb_data, three versions or put the % of shrink

## 6.- XGBoost
- The most accurate modeling technique for structured data.
- Optimize models with __gradient boosting__, state-of-the-art results on a vaiety of datasets.

### Intro (ML is an iterative process)
- Random Forest is an __ensemble method__ cause combine predictions of several modls, in this case several trees.

## Gradient Boosting (another ensemble method)
- Is a method tha goes through cycles to iteratively add models into an ensemble.
- The cycle: naive model (the start) -> mk predictions -> calc loss -> train new model (use loss funct. to fit new mod. -use gradient descent on the los funct. to determine the params. in this new model). -> add new model to the ensemble and -> again mk predics...


### Example

In [13]:
### first part: the beginning of ML Case

import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('files/melb_data.csv')

# Select target and predictors (cols_to_use = preditctors_cols_to_use)
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
y = data.Price
X = data[cols_to_use]

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In this example, you'll work with the XGBoost library. XGBoost stands for extreme gradient boosting, which is an implementation of gradient boosting with several additional features focused on performance and speed. (Scikit-learn has another version of gradient boosting, but XGBoost has some technical advantages.)

In the next code cell, we import the scikit-learn API for XGBoost (xgboost.XGBRegressor). This allows us to build and fit a model just as we would in scikit-learn. As you'll see in the output, the XGBRegressor class has many tunable parameters -- you'll learn about those soon!

In [14]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

We also make predictions and evaluate the model.

In [15]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " +
      str(mean_absolute_error(predictions, y_valid)))
mae = mean_absolute_error(y_valid, predictions)
print(f'XGBRegressor w/o parameter tuning MAE: {mae:,.2f}')

Mean Absolute Error: 249700.73615150957
XGBRegressor w/o parameter tuning MAE: 249,700.74


### Parameter Tuning

XGBoost has a few parameters that can dramatically affect accuracy and training speed. The first parameters you should understand are: *n_estimators*

n_estimators specifies how many times to go through the modeling cycle described above. It is equal to the number of models that we include in the ensemble.
- Too low a value causes underfitting, which leads to inaccurate predictions on both training data and test data.
- Too high a value causes overfitting, which causes accurate predictions on training data, but inaccurate predictions on test data (which is what we care about).

Typical values range from 100-1000, though this depends a lot on the learning_rate parameter discussed below.

Here is the code to set the number of models in the ensemble:

In [16]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [17]:
predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " +
      str(mean_absolute_error(predictions, y_valid)))
mae = mean_absolute_error(y_valid, predictions)
print(f'XGBRegressor w n_estimators=500 MAE: {mae:,.2f}')

Mean Absolute Error: 261610.58339239692
XGBRegressor w n_estimators=500 MAE: 261,610.58


early_stopping_rounds

early_stopping_rounds offers a way to automatically find the ideal value for n_estimators. Early stopping causes the model to stop iterating when the validation score stops improving, even if we aren't at the hard stop for n_estimators. It's smart to set a high value for n_estimators and then use early_stopping_rounds to find the optimal time to stop iterating.

Since random chance sometimes causes a single round where validation scores don't improve, you need to specify a number for how many rounds of straight deterioration to allow before stopping. Setting early_stopping_rounds=5 is a reasonable choice. In this case, we stop after 5 straight rounds of deteriorating validation scores.

When using early_stopping_rounds, you also need to set aside some data for calculating the validation scores - this is done by setting the eval_set parameter.

We can modify the example above to include early stopping:

In [18]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)
# look at early_stopping_rounds and n_estimators values in the output

c:\Users\jm\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [19]:
### JM to mk modification for early_stopping_round deprecated in fit
#`early_stopping_rounds` in `fit` method is deprecated for better compatibility
# with scikit-learn, use `early_stopping_rounds` in constructor or`set_params`

my_mod1 = XGBRegressor(n_estimators=500, early_stopping_rounds=5)
my_mod1.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)
# look at early_stopping_rounds and n_estimators values in the output

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [20]:
predictions = my_mod1.predict(X_valid)
print("Mean Absolute Error: " +
      str(mean_absolute_error(predictions, y_valid)))
mae = mean_absolute_error(y_valid, predictions)
print(f'XGBRegressor(n_estimators=500, early_stopping_rounds=5)\
  MAE: {mae:,.2f}')

Mean Absolute Error: 250196.98340850516
XGBRegressor(n_estimators=500, early_stopping_rounds=5)  MAE: 250,196.98



If you later want to fit a model with all of your data, set n_estimators to whatever value you found to be optimal when run with early stopping.
learning_rate

Instead of getting predictions by simply adding up the predictions from each component model, we can multiply the predictions from each model by a small number (known as the learning rate) before adding them in.

This means each tree we add to the ensemble helps us less. So, we can set a higher value for n_estimators without overfitting. If we use early stopping, the appropriate number of trees will be determined automatically.

In general, a small learning rate and large number of estimators will yield more accurate XGBoost models, though it will also take the model longer to train since it does more iterations through the cycle. As default, XGBoost sets learning_rate=0.1.

Modifying the example above to change the learning rate yields the following code:

In [21]:
my_mod1 = XGBRegressor(n_estimators=1000, early_stopping_rounds=5,
                       learning_rate=0.05)
my_mod1.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)
# see learning_rate value

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [22]:
predictions = my_mod1.predict(X_valid)
print("Mean Absolute Error: " +
      str(mean_absolute_error(predictions, y_valid)))
mae = mean_absolute_error(y_valid, predictions)
print(f'XGBRegressor(n_estimators=1000, early_stopping_rounds=5,\
 learning_rate=0.05) MAE: {mae:,.2f}')

Mean Absolute Error: 248203.45040500737
XGBRegressor(n_estimators=1000, early_stopping_rounds=5, learning_rate=0.05) MAE: 248,203.45


__*n_jobs*__: only to decrease runtime in large datasets

On larger datasets where runtime is a consideration, you can use parallelism to build your models faster. It's common to set the parameter n_jobs equal to the number of cores on your machine. On smaller datasets, this won't help.

The resulting model won't be any better, so micro-optimizing for fitting time is typically nothing but a distraction. But, it's useful in large datasets where you would otherwise spend a long time waiting during the fit command.

Here's the modified example:

In [23]:
my_mod1 = XGBRegressor(n_estimators=1000, early_stopping_rounds=5,
                       learning_rate=0.05, n_jobs=6)
my_mod1.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)
# see learning_rate value

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=6, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [24]:
predictions = my_mod1.predict(X_valid)
print("Mean Absolute Error: " +
      str(mean_absolute_error(predictions, y_valid)))
mae = mean_absolute_error(y_valid, predictions)
print(f'XGBRegressor(+n_jobs=6) MAE: {mae:,.2f}')

Mean Absolute Error: 248203.45040500737
XGBRegressor(+n_jobs=6) MAE: 248,203.45


### Conclusion

XGBoost is a leading software library for working with standard tabular data (the type of data you store in Pandas DataFrames, as opposed to more exotic types of data like images and videos). With careful parameter tuning, you can train highly accurate models.